In [1]:
from abc import ABCMeta, abstractmethod
import datetime
import os
import os.path

import numpy as np
import pandas as pd

# from event import MarketEvent


class DataHandler(object):
    """
    DataHandler is an abstract base class providing an interface for
    all subsequent (inherited) data handlers (both live and historic).

    The goal of a (derived) DataHandler object is to output a generated
    set of bars (OHLCVI) for each symbol requested.

    This will replicate how a live strategy would function as current
    market data would be sent "down the pipe". Thus a historic and live
    system will be treated identically by the rest of the backtesting suite.
    """

    __metaclass__ = ABCMeta

    @abstractmethod
    def get_latest_bars(self, symbol, N=1):
        """
        Returns the last N bars updated.
        """
        raise NotImplementedError("Should implement get_latest_bars()")

    @abstractmethod
    def update_bars(self):
        """
        Pushes the latest bars to the bars_queue for each symbol
        in a tuple OHLCVI format: (datetime, open, high, low,
        close, volume, open interest).
        """
        raise NotImplementedError("Should implement update_bars()")


class HisDataHandler(DataHandler):
    """
    processing historical data from varies source with same methods.
    """

    def __init__(self, events, symbol_list):
        """
        Initialises the historic data handler by requesting the list of symbols.

        Parameters:
        events - The Event Queue.
        symbol_list - A list of symbol strings.
        symbol_data - A generator with structure of {symbolA: DataFrameA, symbolB: DataFrameB,..}
        """
        self.events = events
        self.symbol_list = symbol_list

        self.symbol_data = {}
        self.latest_symbol_data = {}
        self.continue_backtest = True
        self.bar_index = 0

    def symbol_data_from_csv(self, csv_dir,start_time=None, end_time=None):
        """
        :param csv_dir: directory of CSV files;
        :param start_time: time to start backtest;
        :param end_time: time to end backtest
        :return: self.symbol_data
        """
        comb_index = None
        for s in self.symbol_list:
            # Load the CSV file with no header information, indexed on date
            self.symbol_data[s] = pd.read_csv(
                os.path.join(csv_dir, '%s.csv' % s),
                header=0, index_col=0, parse_dates=True,
                names=[
                    'datetime', 'open', 'high',
                    'low', 'close', 'volume'
                ]
            )

            # Combine the index to pad forward values
            if comb_index is None:
                comb_index = self.symbol_data[s].index
            else:
                comb_index.union(self.symbol_data[s].index)

            # Set the latest symbol_data to None
            self.latest_symbol_data[s] = []

        # Reindex the dataframes
        for s in self.symbol_list:
            self.symbol_data[s] = self.symbol_data[s].reindex(index=comb_index, method='pad')
            # slice comb_index with start_time and end_time
            if start_time:
                try:
                    self.symbol_data[s] = self.symbol_data[s][start_time:]
                except KeyError:
                    print('No historical data for start time')
            if end_time:
                try:
                    self.symbol_data[s] = self.symbol_data[s][:end_time]
                except KeyError:
                    print('No historical data for end time')
            self.symbol_data[s] = self.symbol_data[s].iterrows()

    def symbol_data_from_oracle(self, start_time=None, end_time=None):
        """
        :param start_time: time to start backtest;
        :param end_time: time to end backtest
        :return: self.symbol_data
        """
        pass

    def symbol_data_from_wind(self, start_time=None, end_time=None):
        """
        :param start_time: time to start backtest;
        :param end_time: time to end backtest
        :return: self.symbol_data
        """
        from WindPy import w
        w.start()
        for s in self.symbol_list:
            # import data from wind
            wsd_data = w.wsd(s, "open,high,low,close,volume",
                             start_time, end_time, "Fill=Previous")
            df = pd.DataFrame(wsd_data.Data,
                              index=wsd_data.Fields,
                              columns=pd.to_datetime(wsd_data.Times)
                              ).T
            df.columns = ['open', 'high', 'low', 'close', 'volume']
            df.index.name = 'datetime'
            self.symbol_data[s] = df
            self.latest_symbol_data[s] = []

In [2]:
events=[]

In [3]:
dh=HisDataHandler(events,['000300.SH', '000016.SH'])

In [4]:
dh.symbol_data_from_wind(start_time='2017-10-31')

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [5]:
dh_gen=dh.symbol_data['000300.SH'].iterrows()

In [11]:
a[0]

Timestamp('2017-10-31 00:00:00')

In [6]:
a=next(dh_gen)

In [108]:
a[1]['volume']

12105633500.0

In [ ]:
index=datetime()